In [58]:

from functools import partial
from tensorflow.keras.datasets import mnist
from tensorflow import Tensor, math, shape, ones, constant, cast, float32
from tensorflow.keras.layers import Conv2D, Dense, Input, ReLU, concatenate, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.preprocessing import LabelBinarizer

In [59]:
class DP_Cnn:
    def __init__(self, cls_num: int, input_shape: tuple):
        self._cls_num = cls_num
        self._input_shape = input_shape

    def get_dp_cnn(self):
        x = Input(shape=self._input_shape)
        # 双路第一路
        y1 = Conv2D(filters=30, kernel_size=(7, 7), padding="same", strides=2)(x)
        y1 = ReLU()(y1)
        y1 = Conv2D(filters=30, kernel_size=(5, 5), padding="same", strides=2)(y1)
        y1 = ReLU()(y1)
        # 双路第二路
        y2 = Conv2D(filters=30, kernel_size=(3, 3), padding="same", strides=2)(x)
        y2 = ReLU()(y2)
        y2 = Conv2D(filters=30, kernel_size=(3, 3), padding="same", strides=2)(y2)
        y2 = ReLU()(y2)

        # 合并
        y = concatenate([y1, y2], axis=-1)  # 沿通道方向合并

        # 分类
        for filters in [30, 30]:
            y = Conv2D(filters=filters, kernel_size=(3, 3), padding="same", strides=2)(y)
            y = ReLU()(y)
        y = Flatten()(y)
        y = Dense(1000)(y)
        y = Dense(self._cls_num, activation="softmax")(y)
        return Model(x, y)

In [60]:
dp_cnn = DP_Cnn(10, (28, 28, 1)).get_dp_cnn()
dp_cnn.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_36 (Conv2D)              (None, 14, 14, 30)   1500        input_7[0][0]                    
__________________________________________________________________________________________________
conv2d_38 (Conv2D)              (None, 14, 14, 30)   300         input_7[0][0]                    
__________________________________________________________________________________________________
re_lu_36 (ReLU)                 (None, 14, 14, 30)   0           conv2d_36[0][0]                  
____________________________________________________________________________________________

In [61]:

def dp_cnn_loss(label: Tensor, y: Tensor, r: float):
    # loss = cast(ones(shape=(shape(label)[0], 1)) * r, float32)
    marks = cast(math.reduce_sum(math.abs(label - y), axis=-1) > r, float32)
    select_loss = math.reduce_sum(-1. * math.log(1. - math.abs(label - y)), axis=-1) / cast(shape(y)[0], float32)
    return math.reduce_sum(select_loss * marks + (1. - marks) * r)

In [62]:
label = constant([[1., 0.], [0., 1.]])
#
# # loss = ones(shape=(shape(label)[0], 1)) * 0.1
yy = constant([[0.7, 0.3], [0.5, 0.5]])
print(dp_cnn_loss(label, yy, 0.6))
# loss = loss.numpy()
# print(math.reduce_sum(math.abs(label - y), axis=-1))
# loss[math.reduce_sum(math.abs(label - y), axis=-1).numpy() > 0.6] = 10.0
# print(loss)

tf.Tensor(0.6465736, shape=(), dtype=float32)


In [63]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = (x_train / 255.).astype('float32')
x_test = (x_test / 255.).astype('float32')

lb = LabelBinarizer()
y_train = lb.fit_transform(y_train).astype('float32')
y_test = lb.transform(y_test).astype('float32')
print(y_train.dtype)

float32


In [64]:
opt = SGD(learning_rate=0.005)
new_loss = partial(dp_cnn_loss, r=0.01)
dp_cnn.compile(optimizer=opt, loss=new_loss, metrics=['acc'])

In [65]:
dp_cnn.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
734/938 [======================>.......] - ETA: 1s - loss: 0.0508 - acc: 0.0920

KeyboardInterrupt: 